Creiamo delle reti fully connceted per la classificazione degli elettroni vs protoni a bassa energia per Hepd. Vogliamo provare ad avere un campione di elttroni pulito da questi eventi protonici che mimano nel detector degli eventi elettronici rilasciando energia nei primi layer.
Abbiamo già tentato con BDT,CNN e fcnn ora proviamo con reti fully connected con geometrie particolari: proviamo con imbuti o piramidi (partire stretti e poi allergarsi o viceversa). Non abbiamo grandi speranze.

In [1]:
import numpy as np
import os

import warnings

from keras import Model
from keras.layers import Input, Dense,Dropout

from keras.utils import plot_model

creaimo un segmento di codice che aggiunga layer densi con numero di neuroni che si incrementa o che diminuisce in maniera lineare

In [5]:
'''creiamo la funzione che riceva in input:
-num_layers il numero di layer densi da aggiungere
-neurons_init il numero iniziale di neuroni nel layer
-neurons_fin il numero finale di neuroni
-pace il numero di layer da aggiungere prima di modificare il numero di neuroni
'''
class dense_block_lin:
    def __init__(self):
        self.delta=0
    def block_creation(self,x,num_layers=5,neurons_init=32,neurons_fin=512,pace=1):
        #curr: int numero di neuroni corrente
        curr=neurons_init
        #delta: int numero di neuroni da aggiungere ogni pace layers
        if num_layers!=1:
            self.delta=int(((neurons_fin-neurons_init)/(num_layers-1)))*pace
        else:
            delta=0
        for i in range(num_layers-1):
            if i!=0 and i%pace==0:
                curr+=self.delta
            x=Dense(curr,activation='relu')(x)
        x=Dense(neurons_fin,activation='relu')(x)
        return x    

In [2]:
'''
deiniamo una funzione in grado di approssimare il numero totale di paramteri alle migliaia:

1400 -> 1000
1500 -> 2000

12400 -> 12000
12500 -> 13000

100400 -> 100000 
100500 -> 101000
'''

def approssima_alle_migliaia(numero):
    return int(np.ceil((numero/1000)-0.499)*1000)

In [ ]:
#### Creaimo le reti per i dati normalizzati, quindi la dimensione in input è di 40
depths=[2,3,5,7,8,9,11,12,14,16,18]
neur_low=16
high_neurons=[32,54,64,94,128,170,210,256,320,410,512]
i=0
#### NOTA BENE PER LE ARCHITETTURE FUNNEL E BOTTLE NON CONSIDERIAMO high_neurons=32 CON DEPTH 20 PERCHè IL MODELLO RISULTANTE 
#### AVREBBE IN REALTà UN'ARCHITETTURA CONSTANT
for depth in depths:
    for neur_high in high_neurons:
        #de-commenta la riga successiva per creare i modelli bottle 
        #neurons=[neur_low,neur_high]
        #de-commenta la riga successiva per creare i 
        #neurons=[neur_high,neur_low]
        
        #### DOBBIAMO EVITARE DI CREARE IL MODELLO CON ULTIMO LAYER DA 32 E DEPTH DA 20 SE L'ARCHITETTURA DA CREARE NON è CONSTANT 
        #### COMMENTA LE PROSSIMA DUE RIGHE E DE-COMMENTA LA RIGA SEGUENTE PER I MODELLI CONSTANT
        
        #if neur_high==32 and depth==18: continue
        #else:
            neurons=[neur_high,neur_high]
            input_dim=(40,)
            input_layer=Input(shape=input_dim)
            dense_block=dense_block_lin()
            central_block=dense_block.block_creation(input_layer,num_layers=depth,neurons_init=neurons[0],neurons_fin=neurons[1])
            output=Dense(1,activation='sigmoid')(central_block)
            model=Model(input_layer,output)
            print('numero di parametri: ',model.count_params())
            print('numero di layers: ',len(model.layers))
            print('numero di neuroni del layer più largo: ',neur_high)
            
            warnings.filterwarnings('ignore')
            
            ### DEFINIAMO IL TIPO DI ARCHITETTURA IN BASE ALLA VARIAZIONE DELLA LARGHEZZA DEI LAYER NEL NETWORK:
            # bottle : larghezza che aumenta
            # funnel : larghezza che diminuisce (imbuto)
            # constant : larghezza costante
            # general : larghezza senza andamento fisso
            architecture='general'
            if dense_block.delta>0:
                architecture='bottle'
            elif dense_block.delta<0:
                architecture='funnel'
            elif dense_block.delta==0:
                architecture='constant'
            
            print(architecture)
            print(i)
            i+=1
            
            tot_par=approssima_alle_migliaia(model.count_params())
            num_layers=len(model.layers)
            N=len(str(tot_par))-3
            str_par=str(tot_par)[:N]+'k'
            str_num_layers='depth_'+str(num_layers)
            modello_json = model.to_json()
            
            '''
            Salviamo l modello. Nella cartella fcNN troveremo una cartella per modello con il nome simile a:
            
            depth_11_201k :   in questa cartella troveremo un modello di profondità 11 e 201mila parametri, in particolare:
            
            - depth_11_201k_plot.png    : plot del modello
            - depth_11_201k_summary.txt : summary del modello
            - depth_11_201k.json        : modello pre-trainato
            '''
            
            if not os.path.exists('/home/private/Hepd/Dataset_4/fcNN/'+architecture+'/'+architecture+'_'+str_num_layers+'_'+str(neur_high)):
                os.makedirs('/home/private/Hepd/Dataset_4/fcNN/'+architecture+'/'+architecture+'_'+str_num_layers+'_'+str(neur_high))
                    
            with open('/home/private/Hepd/Dataset_4/fcNN/'+architecture+'/'+architecture+'_'+str_num_layers+'_'+str(neur_high)+'/'+
                      architecture+'_'+str_num_layers+'_'+str(neur_high)+'.json', 'w') as f:
                f.write(modello_json)
            #model.summary()
            with open('/home/private/Hepd/Dataset_4/fcNN/'+architecture+'/'+architecture+'_'+str_num_layers+'_'+str(neur_high)+'/'+
                      architecture+'_'+str_num_layers+'_'+str(neur_high)+'_summary.txt', 'w') as f:
                model.summary(print_fn=lambda x: f.write(x + '\n'))
            plot_model(model, to_file='/home/private/Hepd/Dataset_4/fcNN/'+architecture+'/'+architecture+'_'+str_num_layers+'_'+str(neur_high)+'/'+
            architecture+'_'+str_num_layers+'_'+str(neur_high)+'_plot.png',show_shapes=True, show_layer_names=True)

In [8]:
#### Creaimo le reti per i dati PCA, quindi la dimensione in input è minore di 40
#### scegliamo il numero di dimensioni della PCA da sottomettere
input_dimension= 4
depths=[2,3,5,7,8,9,11,12,14,16,18]
neur_low=16
high_neurons=[32,54,64,94,128,170,210,256,320,410,512]
i=0
#### NOTA BENE PER LE ARCHITETTURE FUNNEL E BOTTLE NON CONSIDERIAMO high_neurons=32 CON DEPTH 20 PERCHè IL MODELLO RISULTANTE 
#### AVREBBE IN REALTà UN'ARCHITETTURA CONSTANT
for depth in depths:
    for neur_high in high_neurons:
        #de-commenta la riga successiva per creare i modelli bottle 
        #neurons=[neur_low,neur_high]
        #de-commenta la riga successiva per creare i 
        #neurons=[neur_high,neur_low]
        
        #### DOBBIAMO EVITARE DI CREARE IL MODELLO CON ULTIMO LAYER DA 32 E DEPTH DA 20 SE L'ARCHITETTURA DA CREARE NON è CONSTANT 
        #### COMMENTA LE PROSSIME DUE RIGHE E DE-COMMENTA LA RIGA SEGUENTE PER I MODELLI CONSTANT
        
        #if neur_high==32 and depth==18: continue
        #else:
            neurons=[neur_high,neur_high]
            input_dim=(input_dimension,)
            input_layer=Input(shape=input_dim)
            dense_block=dense_block_lin()
            central_block=dense_block.block_creation(input_layer,num_layers=depth,neurons_init=neurons[0],neurons_fin=neurons[1])
            output=Dense(1,activation='sigmoid')(central_block)
            model=Model(input_layer,output)
            print('numero di parametri: ',model.count_params())
            print('numero di layers: ',len(model.layers))
            print('numero di neuroni del layer più largo: ',neur_high)
            
            warnings.filterwarnings('ignore')
            
            ### DEFINIAMO IL TIPO DI ARCHITETTURA IN BASE ALLA VARIAZIONE DELLA LARGHEZZA DEI LAYER NEL NETWORK:
            # bottle : larghezza che aumenta
            # funnel : larghezza che diminuisce (imbuto)
            # constant : larghezza costante
            # general : larghezza senza andamento fisso
            architecture='general'
            if dense_block.delta>0:
                architecture='bottle'
            elif dense_block.delta<0:
                architecture='funnel'
            elif dense_block.delta==0:
                architecture='constant'
            
            print(architecture)
            print(i)
            i+=1
            
            tot_par=approssima_alle_migliaia(model.count_params())
            num_layers=len(model.layers)
            N=len(str(tot_par))-3
            str_par=str(tot_par)[:N]+'k'
            str_num_layers='depth_'+str(num_layers)
            modello_json = model.to_json()
            
            '''
            Salviamo l modello. Nella cartella fcNN troveremo una cartella per modello con il nome simile a:
            
            depth_11_201k :   in questa cartella troveremo un modello di profondità 11 e 201mila parametri, in particolare:
            
            - depth_11_201k_plot.png    : plot del modello
            - depth_11_201k_summary.txt : summary del modello
            - depth_11_201k.json        : modello pre-trainato
            '''
            
            if not os.path.exists('/home/private/Hepd/Dataset_4/fcNN/PCA/'+architecture+'/'+architecture+'_PCA_'+str_num_layers+'_'+str(neur_high)):
                os.makedirs('/home/private/Hepd/Dataset_4/fcNN/PCA/'+architecture+'/'+architecture+'_PCA_'+str_num_layers+'_'+str(neur_high))
                    
            with open('/home/private/Hepd/Dataset_4/fcNN/PCA/'+architecture+'/'+architecture+'_PCA_'+str_num_layers+'_'+str(neur_high)+'/'+
                      architecture+'_PCA_'+str_num_layers+'_'+str(neur_high)+'.json', 'w') as f:
                f.write(modello_json)
            #model.summary()
            with open('/home/private/Hepd/Dataset_4/fcNN/PCA/'+architecture+'/'+architecture+'_PCA_'+str_num_layers+'_'+str(neur_high)+'/'+
                      architecture+'_PCA_'+str_num_layers+'_'+str(neur_high)+'_summary.txt', 'w') as f:
                model.summary(print_fn=lambda x: f.write(x + '\n'))
            plot_model(model, to_file='/home/private/Hepd/Dataset_4/fcNN/PCA/'+architecture+'/'+architecture+'_PCA_'+str_num_layers+'_'+str(neur_high)+'/'+
            architecture+'_PCA_'+str_num_layers+'_'+str(neur_high)+'_plot.png',show_shapes=True, show_layer_names=True)

numero di parametri:  1249
numero di layers:  4
numero di neuroni del layer più largo:  32
constant
0
numero di parametri:  3295
numero di layers:  4
numero di neuroni del layer più largo:  54
constant
1
numero di parametri:  4545
numero di layers:  4
numero di neuroni del layer più largo:  64
constant
2
numero di parametri:  9495
numero di layers:  4
numero di neuroni del layer più largo:  94
constant
3
numero di parametri:  17281
numero di layers:  4
numero di neuroni del layer più largo:  128
constant
4
numero di parametri:  30091
numero di layers:  4
numero di neuroni del layer più largo:  170
constant
5
numero di parametri:  45571
numero di layers:  4
numero di neuroni del layer più largo:  210
constant
6
numero di parametri:  67329
numero di layers:  4
numero di neuroni del layer più largo:  256
constant
7
numero di parametri:  104641
numero di layers:  4
numero di neuroni del layer più largo:  320
constant
8
numero di parametri:  170971
numero di layers:  4
numero di neuroni del